# 학생 이동 및 교환 비용 행렬 계산

In [1]:
import pandas as pd
df = pd.read_excel("최종_그룹_배정_결과.xlsx", sheet_name="신규반번호순")
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_우울_T점수,정서적 민감성_우울_수준,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,45,보통,48,보통,32,낮음,62,높음,1,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,2
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,3
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,4
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,5


In [2]:
# 초기 그룹 컬럼 추가하기 (랜덤 숫자 기반)
import numpy as np
df['초기그룹'] = np.random.randint(0, 11, size=len(df))
df['상담필요'] = np.random.randint(0, 2, size=len(df))
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호,초기그룹,상담필요
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,48,보통,32,낮음,62,높음,1,1,2,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,2,7,1
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,3,8,1
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,4,7,0
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,5,5,0


In [3]:
def array_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    이산형 변수의 ideal 대비 actual 차이(diff)와 방향(sign)을 계산하여
    3차원 numpy 배열로 반환하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    selected_discrete_variable : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    diff_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      diff_matrix[g, v, c] = 그룹 g, 변수 v, 카테고리 c 의 diff 값
      diff = ideal_count - actual_count

    sign_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      sign_matrix[g, v, c] = diff의 부호 (+1, 0, -1)

    Notes
    -----
    해당 함수는 기존 딕셔너리 구조를 행렬 구조로 바꿔 compute_discrete_cost 계산을 벡터화하기 위해 수정함
    모든 변수중 가장 많은 카테고리 수에 맞춰 0으로 패딩하여 shape이 동일하도록 구성함
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np
    # 확인해보니 반환값이 그룹 번호 순으로 안나와서 정렬 추가
    groups = sorted(list(group_freq.keys()))
    n_groups = len(groups)
    vars_list = selected_discrete_variable
    n_vars = len(vars_list)
    var_to_idx = {var: idx for idx, var in enumerate(vars_list)} # 예시 {성별: 0, 상담필요: 1}
    category_to_idx = {}
    max_categories = 0
    for var in vars_list:
        categories = set(ideal_freq['population'][var].keys()) # 예시 var 성별이면 {1,2}, 상담필요면 {0,1}
        categories = sorted(list(categories))
        category_to_idx[var] = {cat: idx for idx, cat in enumerate(categories)} # 예시 {성별: {1:0, 2:1}, 상담필요: {0:0, 1:1}}
        if len(categories) > max_categories:
            max_categories = len(categories) # 가장 많은 카테고리 수 갱신
    # 행렬 초기화
    diff_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    sign_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    # 행렬 채우기
    try:
        for g_idx, g in enumerate(groups):
            for var in vars_list:
                v_idx = var_to_idx[var]
                cats = category_to_idx[var]
                for cat, c_idx in cats.items():
                    ideal_count = ideal_freq['population'][var].get(cat, 0)
                    acutual_count = group_freq[g][var].get(cat, 0)
                    diff = ideal_count - acutual_count
                    diff_matrix[g_idx, v_idx, c_idx] = diff
                    sign_matrix[g_idx, v_idx, c_idx] = np.sign(diff)
    except Exception as e:
        print("Error during array_diff_and_sign execution:", str(e))
        raise e
    return diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups

In [4]:
# ideal_freq, group_freq, selected_discrete_variable
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
ideal_freq = compute_ideal_discrete_freq(df, ['성별', '상담필요'])
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
print("Ideal Frequency:", ideal_freq)
print("Group Frequency:", group_freq)

Ideal Frequency: {'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
Group Frequency: {2: {'성별': {1: 13, 2: 10}, '상담필요': {0: 12, 1: 11}}, 7: {'성별': {2: 18, 1: 14}, '상담필요': {1: 18, 0: 14}}, 8: {'성별': {2: 15, 1: 12}, '상담필요': {0: 16, 1: 11}}, 5: {'성별': {1: 18, 2: 10}, '상담필요': {0: 15, 1: 13}}, 4: {'성별': {1: 12, 2: 8}, '상담필요': {0: 12, 1: 8}}, 0: {'성별': {2: 13, 1: 12}, '상담필요': {1: 13, 0: 12}}, 10: {'성별': {2: 19, 1: 18}, '상담필요': {1: 20, 0: 17}}, 1: {'성별': {1: 16, 2: 11}, '상담필요': {1: 20, 0: 7}}, 3: {'성별': {2: 18, 1: 8}, '상담필요': {0: 15, 1: 11}}, 9: {'성별': {1: 11, 2: 10}, '상담필요': {0: 16, 1: 5}}, 6: {'성별': {2: 14, 1: 13}, '상담필요': {1: 15, 0: 12}}}


In [5]:
diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups = array_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
print(diff_matrix) # 그룹별 변수별 카테고리별 diff 값
print(sign_matrix) # 그룹별 변수별 카테고리별 sign 값
print(var_to_idx) # 변수명과 인덱스 매핑
print(category_to_idx) # 변수별 카테고리명과 인덱스 매핑
print(groups) # 그룹 번호 리스트

[[[ 1.36363636  0.27272727]
  [ 1.45454545  0.18181818]]

 [[-2.63636364  2.27272727]
  [ 6.45454545 -6.81818182]]

 [[ 0.36363636  3.27272727]
  [ 1.45454545  2.18181818]]

 [[ 5.36363636 -4.72727273]
  [-1.54545455  2.18181818]]

 [[ 1.36363636  5.27272727]
  [ 1.45454545  5.18181818]]

 [[-4.63636364  3.27272727]
  [-1.54545455  0.18181818]]

 [[ 0.36363636 -0.72727273]
  [ 1.45454545 -1.81818182]]

 [[-0.63636364 -4.72727273]
  [-0.54545455 -4.81818182]]

 [[ 1.36363636 -1.72727273]
  [-2.54545455  2.18181818]]

 [[ 2.36363636  3.27272727]
  [-2.54545455  8.18181818]]

 [[-4.63636364 -5.72727273]
  [-3.54545455 -6.81818182]]]
[[[ 1.  1.]
  [ 1.  1.]]

 [[-1.  1.]
  [ 1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1.  1.]
  [-1.  1.]]

 [[ 1. -1.]
  [ 1. -1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1. -1.]
  [-1.  1.]]

 [[ 1.  1.]
  [-1.  1.]]

 [[-1. -1.]
  [-1. -1.]]]
{'성별': 0, '상담필요': 1}
{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}
[0, 1, 2, 3,

In [6]:
# 기존 딕셔너리로 계산하는 함수 가지고 와서 array_diff_and_sign와 동일한지 확인
import pprint
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
pprint.pprint(group_freq)
print(ideal_freq)
def compute_group_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    각 그룹별 이산형 변수의 차이(diff)와 방향(sign)을 계산하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    discrete_vars : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    group_diff_cost : dict
        각 그룹의 변수별 diff와 sign을 담은 딕셔너리
        구조 예시:
        {
            0: {
                '성별_명렬표_diff': {1: 7.5, 2: 9.5},
                '성별_명렬표_sign': {1: +1.0, 2: +1.0},
                '상담 필요_diff': {0: 15.0, 1: 2.0},
                '상담 필요_sign': {0: +1.0, 1: +1.0}
            },
            1: { ... },
            ...
        }

    Notes
    -----
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np

    group_diff_cost = {}
    try:
        for g in group_freq.keys():
            group_diff_cost[g] = {}

            for var in selected_discrete_variable:
                # diff, sign 각각을 별도 dict로 저장
                diff_dict = {}
                sign_dict = {}

                # 각 범주(key)별로 ideal 대비 actual 차이 계산
                for key, ideal_count in ideal_freq['population'][var].items():
                    actual_count = group_freq[g][var].get(key, 0)
                    diff = ideal_count - actual_count
                    diff_dict[key] = diff
                    sign_dict[key] = np.sign(diff) # 이상치보다 작으면 +1, 크면 -1, 같으면 0

                # 변수별 결과를 group_diff_cost에 저장
                group_diff_cost[g][f'{var}_diff'] = diff_dict
                group_diff_cost[g][f'{var}_sign'] = sign_dict
    except Exception as e:
        print("Error during compute_group_diff_and_sign execution:", str(e))
        raise e
    return group_diff_cost
group_diff_cost = compute_group_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
import pprint
pprint.pprint(group_diff_cost)
# 출력값 확인해보니 diff_matrix와 group_diff_cost와 값은 동일하나 그룹 순서가 안맞아서 array_diff_and_sign에 그룹 정렬 추가함
# 행렬의 변수 순서가 다르긴 하지만 이건 var_to_idx로 매핑 가능함

{0: {'상담필요': {0: 12, 1: 13}, '성별': {1: 12, 2: 13}},
 1: {'상담필요': {0: 7, 1: 20}, '성별': {1: 16, 2: 11}},
 2: {'상담필요': {0: 12, 1: 11}, '성별': {1: 13, 2: 10}},
 3: {'상담필요': {0: 15, 1: 11}, '성별': {1: 8, 2: 18}},
 4: {'상담필요': {0: 12, 1: 8}, '성별': {1: 12, 2: 8}},
 5: {'상담필요': {0: 15, 1: 13}, '성별': {1: 18, 2: 10}},
 6: {'상담필요': {0: 12, 1: 15}, '성별': {1: 13, 2: 14}},
 7: {'상담필요': {0: 14, 1: 18}, '성별': {1: 14, 2: 18}},
 8: {'상담필요': {0: 16, 1: 11}, '성별': {1: 12, 2: 15}},
 9: {'상담필요': {0: 16, 1: 5}, '성별': {1: 11, 2: 10}},
 10: {'상담필요': {0: 17, 1: 20}, '성별': {1: 18, 2: 19}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
{0: {'상담필요_diff': {0: 1.454545454545455, 1: 0.18181818181818166},
     '상담필요_sign': {0: 1.0, 1: 1.0},
     '성별_diff': {1: 1.3636363636363633, 2: 0.2727272727272734},
     '성별_sign': {1: 1.0, 2: 1.0}},
 1: {'상담필요_diff': {0: 6.454545454545455, 1: -6.818181818181818},
     '상담필요_sign': {0: 1.0, 1: -1.0},
  

In [7]:
# 출발 그룹 설정이 제대로 되는지 확인
from cost_group_move_v3 import compute_group_total_cost
pprint.pprint(group_freq)
print(ideal_freq)
group_total_cost, _ = compute_group_total_cost(ideal_freq, group_freq, ['성별', '상담필요'])
pprint.pprint(group_total_cost)
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

{0: {'상담필요': {0: 12, 1: 13}, '성별': {1: 12, 2: 13}},
 1: {'상담필요': {0: 7, 1: 20}, '성별': {1: 16, 2: 11}},
 2: {'상담필요': {0: 12, 1: 11}, '성별': {1: 13, 2: 10}},
 3: {'상담필요': {0: 15, 1: 11}, '성별': {1: 8, 2: 18}},
 4: {'상담필요': {0: 12, 1: 8}, '성별': {1: 12, 2: 8}},
 5: {'상담필요': {0: 15, 1: 13}, '성별': {1: 18, 2: 10}},
 6: {'상담필요': {0: 12, 1: 15}, '성별': {1: 13, 2: 14}},
 7: {'상담필요': {0: 14, 1: 18}, '성별': {1: 14, 2: 18}},
 8: {'상담필요': {0: 16, 1: 11}, '성별': {1: 12, 2: 15}},
 9: {'상담필요': {0: 16, 1: 5}, '성별': {1: 11, 2: 10}},
 10: {'상담필요': {0: 17, 1: 20}, '성별': {1: 18, 2: 19}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
{0: 4.082644628099175,
 1: -6.6115702479338765,
 2: 17.719008264462815,
 3: 8.79338842975207,
 4: 58.62809917355372,
 5: -13.140495867768593,
 6: -1.5867768595041305,
 7: -46.26446280991735,
 8: -2.842975206611568,
 9: 76.76033057851241,
 10: -113.35537190082644}
Source Group Index with Minimum Total Cos

In [8]:
# 출발 학생 목록 가지고오기
source_group_df = df[df['초기그룹']==source_group_idx]
print(source_group_df['상담필요'].value_counts())
print(source_group_df['성별'].value_counts())

상담필요
1    20
0    17
Name: count, dtype: int64
성별
2    19
1    18
Name: count, dtype: int64


In [9]:
# 타깃그룹 설정
groups = df['초기그룹'].unique()
target_group_list = [g for g in groups if g != source_group_idx]
print("Target Group List (excluding source group):", target_group_list)

Target Group List (excluding source group): [2, 7, 8, 5, 4, 0, 1, 3, 9, 6]


In [10]:
var_to_idx

{'성별': 0, '상담필요': 1}

In [11]:
category_to_idx

{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}

In [13]:
diff_matrix
# 해당 배열은 아래와 같음
# G, V, C 순서로 되어있음
# G : 그룹 번호
# V : 변수 번호 (0: 성별, 1: 상담필요)
# C : 카테고리 번호 (성별: 0-남,1-여 / 상담필요: 0-필요없음,1-필요함)

array([[[ 1.36363636,  0.27272727],
        [ 1.45454545,  0.18181818]],

       [[-2.63636364,  2.27272727],
        [ 6.45454545, -6.81818182]],

       [[ 0.36363636,  3.27272727],
        [ 1.45454545,  2.18181818]],

       [[ 5.36363636, -4.72727273],
        [-1.54545455,  2.18181818]],

       [[ 1.36363636,  5.27272727],
        [ 1.45454545,  5.18181818]],

       [[-4.63636364,  3.27272727],
        [-1.54545455,  0.18181818]],

       [[ 0.36363636, -0.72727273],
        [ 1.45454545, -1.81818182]],

       [[-0.63636364, -4.72727273],
        [-0.54545455, -4.81818182]],

       [[ 1.36363636, -1.72727273],
        [-2.54545455,  2.18181818]],

       [[ 2.36363636,  3.27272727],
        [-2.54545455,  8.18181818]],

       [[-4.63636364, -5.72727273],
        [-3.54545455, -6.81818182]]])

In [14]:
# 배열의 차원 예시는 아래와 같다
# G : 그룹 번호
diff_matrix[source_group_idx] # 출발 그룹의 변수별 카테고리별 diff 값

array([[-4.63636364, -5.72727273],
       [-3.54545455, -6.81818182]])

In [19]:
# V : 변수 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값
s_row = source_group_df.iloc[0]
print(s_row[['성별', '상담필요']])
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    print(f"Variable: {var}, Index: {v_idx}")
    print(diff_matrix[source_group_idx, v_idx]) # 출발 그룹의 첫번째 변수(성별)의 카테고리별 diff 값

성별      1
상담필요    0
Name: 9, dtype: object
Variable: 성별, Index: 0
[-4.63636364 -5.72727273]
Variable: 상담필요, Index: 1
[-3.54545455 -6.81818182]


In [20]:
# C : 카테고리 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값 중
# 해당 학생이 속한 카테고리의 diff 값
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    cat_value = s_row[var]
    c_idx = category_to_idx[var][cat_value]
    print(f"Variable: {var}, Category Value: {cat_value}, Category Index: {c_idx}")
    print(diff_matrix[source_group_idx, v_idx, c_idx]) # 출발 그룹의 첫번째 변수(성별)의 해당 카테고리의 diff 값

Variable: 성별, Category Value: 1, Category Index: 0
-4.636363636363637
Variable: 상담필요, Category Value: 0, Category Index: 0
-3.545454545454545


In [23]:
# 이동 전 모든 그룹의 diff_matrix 확인
for group_idx in range(diff_matrix.shape[0]):
    print(f"Group Index: {group_idx}")
    print(diff_matrix[group_idx])

Group Index: 0
[[1.36363636 0.27272727]
 [1.45454545 0.18181818]]
Group Index: 1
[[-2.63636364  2.27272727]
 [ 6.45454545 -6.81818182]]
Group Index: 2
[[0.36363636 3.27272727]
 [1.45454545 2.18181818]]
Group Index: 3
[[ 5.36363636 -4.72727273]
 [-1.54545455  2.18181818]]
Group Index: 4
[[1.36363636 5.27272727]
 [1.45454545 5.18181818]]
Group Index: 5
[[-4.63636364  3.27272727]
 [-1.54545455  0.18181818]]
Group Index: 6
[[ 0.36363636 -0.72727273]
 [ 1.45454545 -1.81818182]]
Group Index: 7
[[-0.63636364 -4.72727273]
 [-0.54545455 -4.81818182]]
Group Index: 8
[[ 1.36363636 -1.72727273]
 [-2.54545455  2.18181818]]
Group Index: 9
[[ 2.36363636  3.27272727]
 [-2.54545455  8.18181818]]
Group Index: 10
[[-4.63636364 -5.72727273]
 [-3.54545455 -6.81818182]]


In [ ]:
# 출발 그룹에서 모든 학생들이 도착 그룹으로 이동했을때 diff_matrix 계산
'''
어떻게 생각해야할까?
출발 그룹 중 하나의 학생이
도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?
그면 차원이 어떻게 될까?
    출발 그룹 학생 수 만큼 차원이 늘어나고
    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?
    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?
    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데
'''
# 우선 차원 틀 먼저 만들어보자
# N : 출발 그룹 학생 수
# G : 도착 그룹 수 (출발 그룹 제외해야하지만 계산을 위해 모든 그룹 포함)
# V : 변수 수
# C : 가장 많은 카테고리 수